# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv("weather_data.csv")
del weather_df["Unnamed: 0"]
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,72.54,73,0,9.22,ZA,1645462562
1,Punta Arenas,-53.1500,-70.9167,62.71,45,0,17.27,CL,1645462427
2,Hobart,-42.8794,147.3294,49.86,72,75,5.75,AU,1645462562
3,Esperance,-33.8667,121.9000,62.44,70,0,15.82,AU,1645462562
4,Dunedin,-45.8742,170.5036,58.80,81,1,1.01,NZ,1645462563


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=weather_df[["Lat","Lng"]]
humidity=weather_df["Humidity"]

In [5]:
my_fig=gmaps.figure()
heatmap=gmaps.heatmap_layer(locations, weights=humidity)

my_fig.add_layer(heatmap)
my_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#wish I could distinguish by rain/snow and/or misty conditions
my_weather=weather_df.loc[(
    (weather_df["Cloudiness"]>70) & 
    (weather_df["Max Temp"] <=60) & 
    (weather_df["Country"] != 'US')& 
    (weather_df["Country"] != 'RU')
    )]
my_weather=my_weather.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=my_weather
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Hobart,-42.8794,147.3294,49.86,72,75,5.75,AU,1645462562
7,Gram,55.2895,9.0491,42.84,81,75,13.80,DK,1645462564
16,Gravelbourg,49.8834,-106.5512,-9.13,90,100,6.35,CA,1645462566
19,Nemuro,43.3236,145.5750,22.62,79,100,34.27,JP,1645462567
21,Hong Kong,22.2855,114.1577,52.25,92,100,10.00,HK,1645461785
...,...,...,...,...,...,...,...,...,...
502,Lahuachaca,-17.3667,-67.6667,57.87,46,98,7.34,BO,1645462724
505,Sfântu Gheorghe,45.8667,25.7833,44.82,60,99,0.69,RO,1645462518
508,Leshan,29.5623,103.7639,41.00,90,100,2.55,CN,1645462726
515,Hashtrūd,37.4779,47.0508,23.16,94,100,2.77,IR,1645462728


In [9]:
#lat_lngs=zip(lat,lng)
hotels = []
lat=hotel_df["Lat"]
lng=hotel_df["Lng"]
lat_lngs=zip(lat,lng)

In [10]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
type="lodging"
bounds=5000

for lat_lng in lat_lngs:
    coord=f"{lat_lng[0]},{lat_lng[1]}"
    params={"location": coord,"radius":bounds,"types": type,"key":g_key}
    response=requests.get(url,params=params)
    hotel_data=response.json()
    try:
        hotels.append(hotel_data["results"][1]["name"])
    except:
        print("no hotels nearby...")
        #oops, need to put some type of placeholder for no hotel
        hotels.append("no hotels nearby...")



no hotels nearby...
no hotels nearby...
no hotels nearby...
no hotels nearby...
no hotels nearby...


In [11]:
hotels
#it works!!!

['Mantra on Collins Hobart',
 'Annemettes Bondegård',
 "Ming's bed and breakfast",
 'Restaurant Hotel Lupinus',
 'ibis Hong Kong Central & Sheung Wan',
 'no hotels nearby...',
 'Hotel Angmagssalik',
 'Bensvik Brygge',
 'Holel R',
 'Bradshaw House Bluff',
 'متنزه و بحيرة سيان',
 'Southland Organic Farmstay',
 'Sømandshjemmet',
 'Dee-Jay Motel',
 'Hotel Icefiord',
 'Vivstavarvstjärns Camping',
 'Kashiwaso',
 '富山荘',
 'Graciosa Resort - Biosphere Island Hotel',
 'Mehamn Vandrerhjem',
 '民宿・海の家 若草',
 'Naujaaraaluit – Clyde River Hotel',
 'Kinross Inn',
 'Weinan Taoyuan Meisha International Hotel',
 'no hotels nearby...',
 'Café de Upernavik',
 "Raven's Rest Inn",
 '怡居快捷酒店黄河路店',
 'Höfn Inn Guesthouse',
 'Jinsha Hotel',
 'Tabořiště Pískáč Postřelmov',
 'Grund',
 'Tian Sheng SSAW Boutique Hotel Qi Dong Downtown',
 'Colborne Bed and Breakfast',
 'Accommodation Rokiškyje',
 'City Comfort Inn Laibin Liulai',
 'Wellness Resort Okinawa Vacation Center Yuinchi Hotel Nanjo',
 '交通宾馆二部',
 'Hotel Zamojsk

In [12]:
#add the new column
hotel_df["Hotel Name"]= hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,Hobart,-42.8794,147.3294,49.86,72,75,5.75,AU,1645462562,Mantra on Collins Hobart
7,Gram,55.2895,9.0491,42.84,81,75,13.80,DK,1645462564,Annemettes Bondegård
16,Gravelbourg,49.8834,-106.5512,-9.13,90,100,6.35,CA,1645462566,Ming's bed and breakfast
19,Nemuro,43.3236,145.5750,22.62,79,100,34.27,JP,1645462567,Restaurant Hotel Lupinus
21,Hong Kong,22.2855,114.1577,52.25,92,100,10.00,HK,1645461785,ibis Hong Kong Central & Sheung Wan
...,...,...,...,...,...,...,...,...,...,...
502,Lahuachaca,-17.3667,-67.6667,57.87,46,98,7.34,BO,1645462724,FAMILIA CORONEL
505,Sfântu Gheorghe,45.8667,25.7833,44.82,60,99,0.69,RO,1645462518,Hotel Castel
508,Leshan,29.5623,103.7639,41.00,90,100,2.55,CN,1645462726,乐山名流酒店
515,Hashtrūd,37.4779,47.0508,23.16,94,100,2.77,IR,1645462728,no hotels nearby...


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [6]:
# Add marker layer ontop of heat map
#I could not figure this part out
hotel_fig=gmaps.figure()
markers=gmaps.marker_layer(locations)
marker_layer=gmaps.marker_layer(locations)


hotel_fig.add_layer(marker_layer)
hotel_fig

# Display figure



Figure(layout=FigureLayout(height='420px'))